In [1]:
import torch
# Example tensor on CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using device:',device)

using device: cuda


In [3]:
import pandas as pd
filename = "C://Users//e22cseu0125-legal//Downloads//FD_ind_cs.xlsx"  
df = pd.read_excel(filename)
print(df.head())

   Unnamed: 0                                               data  \
0           0  iminal Appeal No. 47 of 1963. Appeal by specia...   
1           1  Sub section\n(3) of section 198B provides that...   
2           2  Sub section (1) (a) of section 83 states that ...   
3           3  Appellant 's counsel on the other hand contend...   
4           4  10 sleeping under a Neem tree at some distance...   

                                             summary  
0  and in convicting the appellant.The appellant ...  
1  Furthermore, although the impugned article did...  
2  Clause (a) of sub section\n(1) of section 83 s...  
3  Rule 38(1) of the Conduct of Election Rules, 1...  
4  It was the obligation of the presiding officer...  


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import pandas as pd
from torch.utils.data import Dataset
import torch

model_name = "EleutherAI/pythia-160m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


class LegalDataset(Dataset):
    def __init__(self, data_frame, tokenizer, max_length=1024):
        self.data = data_frame
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        source, target = row['data'], row['summary']
        tokenized_input = self.tokenizer(
            source, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        tokenized_output = self.tokenizer(
            target, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        input_ids = tokenized_input["input_ids"].squeeze()
        attention_mask = tokenized_input["attention_mask"].squeeze()
        labels = tokenized_output["input_ids"].squeeze()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
dataset = LegalDataset(df, tokenizer)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

training_args = TrainingArguments(
    output_dir="./pythia_finetuned",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    fp16=True,  # Mixed precision training for faster performance
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained("./pythia_finetuned")
tokenizer.save_pretrained("./pythia_finetuned")


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

C:\Users\e22cseu0125-legal\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\e22cseu0125-legal\.cache\huggingface\hub\models--EleutherAI--pythia-160m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

C:\Users\e22cseu0125-legal\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\e22cseu0125-legal\AppData\Local\Temp\ipykernel_59120\421429253.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.409600,2.601915
2,2.386400,2.621872
3,2.504600,2.587350
4,2.681800,2.684954
5,2.115800,2.702555
6,2.453600,2.778230
7,2.325400,2.873110
8,2.283700,2.918497
9,2.069100,3.076483
10,2.279600,3.309355


('./pythia_finetuned\\tokenizer_config.json',
 './pythia_finetuned\\special_tokens_map.json',
 './pythia_finetuned\\tokenizer.json')

In [7]:
def get_root_path():
    path = "./test-data1"   #CORRECT THIS
    return path

def get_summary_data(dataset, train):

    path = get_root_path()+'/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary


def get_req_len_dict(dataset, istrain):

    f = open(get_root_path() +"/"+istrain+"-data"+"/stats-"+ dataset+"-" + istrain +".txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents



def nest_sentences(document,chunk_length):

    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested



In [9]:
import glob
from nltk import tokenize
import nltk
import transformers
import torch
nltk.download('punkt')
nltk.download('punkt_tab')

dataset = "IN"
names, data_source, data_summary = get_summary_data(dataset, "test")
print(len(names))
print(len(data_source))
print(len(data_summary))
dict_names = get_req_len_dict(dataset, "test")
dict_names

100
100
100
['']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e22cseu0125-
[nltk_data]     legal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\e22cseu0125-
[nltk_data]     legal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


{'4963.txt': 596,
 '5994.txt': 1048,
 '7130.txt': 1374,
 '6118.txt': 569,
 '4860.txt': 826,
 '660.txt': 507,
 '3436.txt': 322,
 '6276.txt': 599,
 '2727.txt': 316,
 '3356.txt': 354,
 '1531.txt': 514,
 '2609.txt': 353,
 '1522.txt': 272,
 '2392.txt': 480,
 '1378.txt': 205,
 '4071.txt': 662,
 '3168.txt': 736,
 '232.txt': 1872,
 '4568.txt': 978,
 '2440.txt': 1166,
 '2913.txt': 488,
 '1697.txt': 1389,
 '3844.txt': 1448,
 '5364.txt': 3302,
 '3542.txt': 266,
 '266.txt': 410,
 '3210.txt': 603,
 '1974.txt': 529,
 '6003.txt': 1571,
 '5141.txt': 919,
 '2207.txt': 717,
 '2593.txt': 630,
 '1195.txt': 531,
 '1406.txt': 410,
 '2627.txt': 752,
 '6270.txt': 421,
 '3924.txt': 769,
 '6778.txt': 10008,
 '2124.txt': 163,
 '78.txt': 398,
 '4316.txt': 1720,
 '1778.txt': 468,
 '314.txt': 258,
 '690.txt': 1066,
 '1789.txt': 346,
 '2122.txt': 3873,
 '2796.txt': 479,
 '6852.txt': 669,
 '2649.txt': 466,
 '6728.txt': 436,
 '2052.txt': 746,
 '5266.txt': 6079,
 '5248.txt': 1089,
 '2304.txt': 855,
 '6881.txt': 854,
 '

In [11]:
def generate_summary_gpu(nested_sentences,p=0.2):

  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    l = int(p * len(nested.split(" ")))
    input_tokenized = tokenizer.encode(nested, truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = model.to(device).generate(input_tokenized,
                                      length_penalty=0.01,
                                      min_length=l-5,
                                      max_length=2048)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [13]:
output_path = "./output_pythia/"
import os
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [15]:
import os

for i in range(len(data_source)):
    name = names[i]
    doc = data_source[i]
    wc = doc.split(" ")
    input_len = len(wc)
        # Extract only the file name (e.g., '1181.txt') from `name`
    base_name = os.path.basename(name)
    
    # Debug to verify
    print(f"Base Name: {base_name}")
    
    # Access the correct key in `dict_names`
    if base_name not in dict_names:
        print(f"KeyError: '{base_name}' not found in dict_names")
        continue  # Skip this iteration if the key is missing
    
    req_len = dict_names[base_name]

    print(str(i) + ": " + name + " - " + str(input_len) + " : " + str(req_len), end=" ,")

    nested = nest_sentences(doc, 1024)
    l = int(req_len / len(nested))
    p = float(req_len / input_len)
    print(p)

    abs_summ = generate_summary_gpu(nested, p)

    abs_summ = " ".join(abs_summ)
    if len(abs_summ.split(" ")) > req_len:
        abs_summ = abs_summ.split(" ")
        abs_summ = abs_summ[:req_len]
        abs_summ = " ".join(abs_summ)

    print(len(abs_summ.split(" ")))

    # Construct the path correctly
    path = os.path.join(output_path, name)
    print(f"Constructed Path: {path}")  # Debug to check the path

    # Normalize the path for consistency
    path = os.path.normpath(path)

    # Debug: Check if the directory exists
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        print(f"Directory does not exist: {directory}")
        os.makedirs(directory)
        print(f"Directory created: {directory}")

    # Debug: Check if the full path exists before opening the file
    if os.path.exists(path):
        print(f"The file already exists: {path}")
    else:
        print(f"The file does not exist: {path}")

    # Write the file
    with open(path, 'w') as file:
        file.write(abs_summ)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\e22cseu0125-legal\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:657: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.01` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Base Name: 1181.txt
0: judgement\1181.txt - 3387 : 321 ,0.09477413640389726


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


321
Constructed Path: ./output_pythia/judgement\1181.txt
Directory does not exist: output_pythia\judgement
Directory created: output_pythia\judgement
The file does not exist: output_pythia\judgement\1181.txt
Base Name: 1195.txt
1: judgement\1195.txt - 4234 : 531 ,0.12541332073689182
531
Constructed Path: ./output_pythia/judgement\1195.txt
The file does not exist: output_pythia\judgement\1195.txt
Base Name: 1329.txt
2: judgement\1329.txt - 2990 : 343 ,0.11471571906354515


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

623
Constructed Path: ./output_pythia/judgement\7109.txt
The file does not exist: output_pythia\judgement\7109.txt
Base Name: 7130.txt
96: judgement\7130.txt - 6171 : 1374 ,0.22265435099659697


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1374
Constructed Path: ./output_pythia/judgement\7130.txt
The file does not exist: output_pythia\judgement\7130.txt
Base Name: 715.txt
97: judgement\715.txt - 1346 : 249 ,0.1849925705794948
249
Constructed Path: ./output_pythia/judgement\715.txt
The file does not exist: output_pythia\judgement\715.txt
Base Name: 78.txt
98: judgement\78.txt - 2035 : 398 ,0.1955773955773956
398
Constructed Path: ./output_pythia/judgement\78.txt
The file does not exist: output_pythia\judgement\78.txt
Base Name: 784.txt
99: judgement\784.txt - 3712 : 463 ,0.12473060344827586


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


463
Constructed Path: ./output_pythia/judgement\784.txt
The file does not exist: output_pythia\judgement\784.txt


In [18]:
import os
from rouge_score import rouge_scorer

# Define paths
original_summaries_dir = './summary'   
generated_summaries_dir = './output_pythia/judgement'

def read_summary(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

original_summaries = {}
generated_summaries = {}

# Process original summaries
for filename in os.listdir(original_summaries_dir):
    # Normalize paths
    original_file_path = os.path.normpath(os.path.join(original_summaries_dir, filename))
    generated_file_path = os.path.normpath(os.path.join(generated_summaries_dir, filename))
    
    # Extract base name without extension
    name = os.path.splitext(filename)[0]

    # Check if the corresponding file exists in generated summaries
    if not os.path.exists(generated_file_path):
        print(f"File not found in generated summaries: {generated_file_path}")
        continue

    # Read files
    original_summaries[name] = read_summary(original_file_path)
    generated_summaries[name] = read_summary(generated_file_path)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to compute ROUGE scores
def compute_rouge(original, generated):
    scores = scorer.score(original, generated)
    return scores

# Compute and display ROUGE scores
for name in original_summaries:
    if name not in generated_summaries:
        print(f"Generated summary missing for: {name}")
        continue

    original = original_summaries[name]
    generated = generated_summaries[name]

    rouge_scores = compute_rouge(original, generated)
    print(f"ROUGE scores for {name}:")
    print(f"ROUGE-1: {rouge_scores['rouge1']}")
    print(f"ROUGE-2: {rouge_scores['rouge2']}")
    print(f"ROUGE-L: {rouge_scores['rougeL']}")
    print()



ROUGE scores for 1181:
ROUGE-1: Score(precision=0.5337423312883436, recall=0.54375, fmeasure=0.5386996904024768)
ROUGE-2: Score(precision=0.19692307692307692, recall=0.2006269592476489, fmeasure=0.19875776397515527)
ROUGE-L: Score(precision=0.2361963190184049, recall=0.240625, fmeasure=0.23839009287925697)

ROUGE scores for 1195:
ROUGE-1: Score(precision=0.4670433145009416, recall=0.4635514018691589, fmeasure=0.4652908067542214)
ROUGE-2: Score(precision=0.14150943396226415, recall=0.1404494382022472, fmeasure=0.14097744360902256)
ROUGE-L: Score(precision=0.18832391713747645, recall=0.18691588785046728, fmeasure=0.18761726078799248)

ROUGE scores for 1329:
ROUGE-1: Score(precision=0.4943181818181818, recall=0.5058139534883721, fmeasure=0.5)
ROUGE-2: Score(precision=0.21652421652421652, recall=0.22157434402332363, fmeasure=0.21902017291066284)
ROUGE-L: Score(precision=0.23295454545454544, recall=0.23837209302325582, fmeasure=0.235632183908046)

ROUGE scores for 1378:
ROUGE-1: Score(preci

In [20]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_rouge(original, generated):
    scores = scorer.score(original, generated)
    return scores

rouge1_f1_total = 0
rouge2_f1_total = 0
rougeL_f1_total = 0
num_summaries = len(original_summaries)

for name in original_summaries:
    original = original_summaries[name]
    generated = generated_summaries[name]

    rouge_scores = compute_rouge(original, generated)

    rouge1_f1_total += rouge_scores['rouge1'].fmeasure
    rouge2_f1_total += rouge_scores['rouge2'].fmeasure
    rougeL_f1_total += rouge_scores['rougeL'].fmeasure

avg_rouge1_f1 = rouge1_f1_total / num_summaries
avg_rouge2_f1 = rouge2_f1_total / num_summaries
avg_rougeL_f1 = rougeL_f1_total / num_summaries

print(f"Average ROUGE-1 F1 score: {avg_rouge1_f1:.4f}")
print(f"Average ROUGE-2 F1 score: {avg_rouge2_f1:.4f}")
print(f"Average ROUGE-L F1 score: {avg_rougeL_f1:.4f}")

Average ROUGE-1 F1 score: 0.5456
Average ROUGE-2 F1 score: 0.2493
Average ROUGE-L F1 score: 0.2465
